In [2]:
%matplotlib inline

import sys
import os
sys.path.append(os.path.abspath('C:/cygwin/home/mramire8/python_code/sr/active'))
sys.path.append(os.path.abspath('/Users/maru/MyCode/sr/active'))

from sklearn import linear_model

from datautil.textutils import StemTokenizer
from datautil.load_data import *

import numpy as np
from time import time
import pickle

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

mpl.style.use('bmh')

## Get the data ready
imdb_path = 'C:/Users/mramire8/Documents/Research/Oracle confidence and Interruption/dataset/aclImdb/raw-data'
# imdb_path = '/Users/maru/MyCode/data/imdb'

categories = [['alt.atheism', 'talk.religion.misc'],
              ['comp.graphics', 'comp.windows.x'],
              ['comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware'],
              ['rec.sport.baseball', 'sci.crypt']]

vct = CountVectorizer(encoding='latin-1', min_df=5, max_df=1.0, binary=True, ngram_range=(1, 3),
                      token_pattern='\\b\\w+\\b', tokenizer=StemTokenizer())

vct2 = CountVectorizer(encoding='latin-1', min_df=5, max_df=1.0, binary=True, ngram_range=(1, 3),
                      token_pattern='\\b\\w+\\b', tokenizer=StemTokenizer())



In [ ]:

dataset = load_dataset("imdb", 100, categories[0], vct, 100, raw=True,  percent=.5, keep_subject=True)
sraa = load_dataset("aviation", 100, categories[0], vct2, 100, raw=True,  percent=.5, keep_subject=True)

# data = load_dataset('imdb', None, categories[0], vct, 100, percent=.5, keep_subject=True)


In [5]:
print("Data size %s" % len(dataset.train.data))
print

kvalues = [10, 25, 50, 75, 100]
cost = np.array([5.7, 8.2, 10.9, 15.9, 16.7])

threshold = .4

def compute_q_max(data_name, dataset, vct, penalty):
    q_max = np.zeros((5,len(dataset.train.data)))
    a_max = np.zeros((5,len(dataset.train.data)))
    neutral =  np.zeros((5,len(dataset.train.data)))
    for ki, fixk in enumerate(kvalues):

        fixk_saved = "{0}{1}.p".format(data_name, fixk)
        try:
            fixk_file = open(fixk_saved, "rb")
            data = pickle.load(fixk_file)
        except IOError:
            data = process_data(dataset, fixk, 100, vct, silent=True)
            fixk_file = open(fixk_saved, "wb")
            pickle.dump(data, fixk_file)

        train_x = data.test.bow
        train_y = data.test.target

        test_x = data.train.bowk
        test_y = data.train.target

        print "*"*60
        print
        print("K= %s" % fixk)


        clf = linear_model.LogisticRegression(penalty='l1', C=penalty)
        print("penalty: %s" % penalty)
        clf.fit(train_x, train_y)


        prob_y = clf.predict_proba(test_x)

        q = prob_y.max(axis=1)

        unc = prob_y.min(axis=1)

        print "Ave. unc: %s" % unc.mean()
        print "Ave. qmax: %s" % q.mean()
        q_max[ki] = q
    return q_max

q_max = compute_q_max('imdb', dataset, vct, 0.3)



Data size 22267

************************************************************

K= 10
penalty: 0.3
Ave. unc: 0.383221396573
Ave. qmax: 0.616778603427
Total Documents: 22267
Minimum size: 100
Fix k: 25
Docs left: 22267
Vectorizing ...
************************************************************

K= 25
penalty: 0.3
Ave. unc: 0.31536401975
Ave. qmax: 0.68463598025
Total Documents: 22267
Minimum size: 100
Fix k: 50
Docs left: 22267
Vectorizing ...
************************************************************

K= 50
penalty: 0.3
Ave. unc: 0.254261074089
Ave. qmax: 0.745738925911
Total Documents: 22267
Minimum size: 100
Fix k: 75
Docs left: 22267
Vectorizing ...
************************************************************

K= 75
penalty: 0.3
Ave. unc: 0.214122086335
Ave. qmax: 0.785877913665
Total Documents: 22267
Minimum size: 100
Fix k: 100
Docs left: 22267
Vectorizing ...
************************************************************

K= 100
penalty: 0.3
Ave. unc: 0.183779531326
Ave. qmax: 0.

In [6]:
q_max.shape

(5L, 22267L)

In [14]:
q_max_sel = q_max.argmax(axis=0)


In [18]:
q_max_sel[:10]

array([3, 2, 4, 3, 2, 2, 2, 4, 4, 2], dtype=int64)

In [16]:
for i in range(5):
    print "\t".join(["{:.3f}".format(a) for a in q_max[i][:10]])

0.731	0.511	0.659	0.505	0.889	0.516	0.603	0.760	0.804	0.631
0.765	0.582	0.623	0.541	0.922	0.655	0.593	0.953	0.880	0.747
0.719	0.726	0.691	0.529	0.934	0.862	0.764	0.988	0.890	0.818
0.798	0.590	0.756	0.652	0.898	0.777	0.718	0.999	0.930	0.770
0.676	0.563	0.779	0.635	0.677	0.522	0.650	0.999	0.981	0.588


In [31]:
# Percentage of 10-word subinstances where Q_max = Q_10
1. * sum((q_max_sel == 0) [(1-q_max[0]) < .4]) / len(q_max_sel) * 100

2.5239143126599903

In [62]:
def fn(x, alpha):
    return np.power(x,alpha)

def fn_obj(x, cost, alpha):
    ''' Objective function  x^alpha / cost '''
    return fn(x,alpha) / cost

def compute_alpha(q_max, alpha, cost):
    q_obj = np.zeros(q_max.shape)
    
    # For every row = subinstance k, compute objective function
    for i in range(len(q_obj)):
        q_obj[i] = fn_obj(q_max[i], cost[i], alpha)
        
    return q_obj

def max_alpha(q_max, alpha, cost):
    q = compute_alpha(q_max, alpha, cost)
    return q.argmax(axis=0)

def stats_k(q_mx_sl, q_alpha_sl):
    ''' Compares two matrix'''

    stats ={}
    stats['match'] = np.mean(q_mx_sl == q_alpha_sl)
    stats['cheaper'] = np.mean(q_mx_sl > q_alpha_sl)
    stats['expensive'] = np.mean(q_mx_sl < q_alpha_sl)
    
    for a in np.unique(q_mx_sl):
        stats['k=%s'%a] = np.mean(q_alpha_sl == a)
    
    
    return stats

def print_stats(stats):
    for k,v in stats.items():
        print "{}:\t{:.4f}".format(k,v)


In [70]:
alphas = np.array([0,1,5,10,100,200,1000,2000])

for a in alphas:
    st = stats_k(q_max_sel, max_alpha(q_max, a, cost))
    print "\n== Alpha: %s ==" % a
    print_stats(st)
    


== Alpha: 0 ==
k=4:	0.0000
k=3:	0.0000
k=2:	0.0000
k=1:	0.0000
k=0:	1.0000
cheaper:	0.9720
match:	0.0280
expensive:	0.0000

== Alpha: 1 ==
k=4:	0.0000
k=3:	0.0000
k=2:	0.0007
k=1:	0.0676
k=0:	0.9317
cheaper:	0.9616
match:	0.0384
expensive:	0.0000

== Alpha: 5 ==
k=4:	0.2691
k=3:	0.0977
k=2:	0.2593
k=1:	0.1949
k=0:	0.1791
cheaper:	0.4673
match:	0.5327
expensive:	0.0000

== Alpha: 10 ==
k=4:	0.3933
k=3:	0.1501
k=2:	0.2329
k=1:	0.1429
k=0:	0.0807
cheaper:	0.2587
match:	0.7413
expensive:	0.0000

== Alpha: 100 ==
k=4:	0.5358
k=3:	0.2003
k=2:	0.1482
k=1:	0.0841
k=0:	0.0316
cheaper:	0.0304
match:	0.9696
expensive:	0.0000

== Alpha: 200 ==
k=4:	0.5452
k=3:	0.2023
k=2:	0.1410
k=1:	0.0818
k=0:	0.0297
cheaper:	0.0159
match:	0.9841
expensive:	0.0000

== Alpha: 1000 ==
k=4:	0.5535
k=3:	0.2034
k=2:	0.1347
k=1:	0.0800
k=0:	0.0285
cheaper:	0.0032
match:	0.9968
expensive:	0.0000

== Alpha: 2000 ==
k=4:	0.5236
k=3:	0.1787
k=2:	0.1102
k=1:	0.0592
k=0:	0.1282
cheaper:	0.1016
match:	0.8984
expensive:	0.00

In [61]:
q_max_sraa = compute_q_max('aviation', dataset, vct2, 0.01)

k=4:		       0.0
k=3:		       0.0
k=2:		       0.0
k=1:		       0.0
k=0:		       1.0
cheaper:		  0.971976
match:		 0.0280235
expensive:		       0.0
